### Single AWE ANcode Decoder A=67 Synthesizer

In [11]:
import math
import numpy as np

def bits_needed(number):
    return number.bit_length()

In [12]:
GF67_positive =  [1, 2, 4, 8, 16, 32, 64, 61, 55, 43, 19, 38, 9, 18, 36, 5, 10, 20, 40, 13, 26, 52, 37, 7, 14, 28, 56, 45, 23, 46, 25, 50, 33]
GF67_negative =  [66, 65, 63, 59, 51, 35, 3, 6, 12, 24, 48, 29, 58, 49, 31, 62, 57, 47, 27, 54, 41, 15, 30, 60, 53, 39, 11, 22, 44, 21, 42, 17, 34]
print(np.size(GF67_positive))
print(np.size(GF67_negative))

33
33


In [16]:
plus_values67 = list(range(1, len(GF67_positive) + 1))
minus_values67 = [-i for i in range(1, len(GF67_negative) + 1)]
print(plus_values67)
print(minus_values67)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33]


### 選取特定的位元數:

In [18]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data = (1 << 24) - 1        ### 24bits
A = 67
input = data * A
bits = bits_needed(input)

print(input)
print("AN資料的位元數:", bits)
print(bits_needed(A))
print(bits_needed(data))
print(bits_needed(bits))


1124073405
AN資料的位元數: 31
7
24
5


In [23]:
#### SEC_AN Decoder
# 2025/5/12
#-------------------------------------
# Used to do Conventional SEC, corrected errors by AWE
# input: remainder, output: AWE
Bw = bits
Bn = bits_needed(data)
BA = bits_needed(A)
B_location = bits_needed(bits)
#-------------------------------------
fn = 'SECdecoder' +"_AWE_"+ str(Bn) + "bits" +'.v'
f = open(fn, 'w')

f.write('// Product (AN) Code Conventional SECdecoder\n')
f.write('// ' + fn + '\n')
f.write('// Used to do SEC by Galois Field, and corrected AWE\n')
f.write('// Received remainder r, output single AWE.\n')
f.write('module SECdecoder_AWE_%dbits(r, AWE);\n'% (Bn))
f.write('input \t[%d:0]\tr;\n' % (BA-1))
f.write('output\treg\tsigned\t[%d:0]\tAWE;\n' % (np.size(GF67_positive)))

f.write('always@(*) begin\n')
f.write('\tcase(r)\n')
for i in range(np.size(GF67_positive)):
    f.write("\t\t%d: AWE = +(1 << %d);\n" % (GF67_positive[i] ,plus_values67[i]-1))
for j in range(np.size(GF67_negative)):
    f.write("\t\t%d: AWE = -(1 << %d);\n" % (GF67_negative[j] ,plus_values67[j]-1))
f.write("\t\tdefault: AWE = 0;\n")
f.write('\tendcase\n')
f.write('end\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file SECdecoder_AWE_24bits.v is generated.



In [24]:
#### SEC_AN Decoder
# 2025/5/12
#-------------------------------------
# Used to do Conventional SEC, corrected errors by location, not AWE
# input: remainder, output: location
Bw = bits
Bn = bits_needed(data)
BA = bits_needed(A)
B_location = bits_needed(bits)
#-------------------------------------
fn = 'SECdecoder' +"_location_"+ str(Bn) + "bits" +'.v'
f = open(fn, 'w')

f.write('// Product (AN) Code Conventional SECdecoder\n')
f.write('// ' + fn + '\n')
f.write('// Used to do SEC by Galois Field, and corrected by location, not AWE\n')
f.write('// Received remainder r, output error location.\n')
f.write('module SECdecoder_location_%dbits(r, l);\n'% (Bn))
f.write('input \t[%d:0]\tr;\n' % (BA-1))
f.write('output\treg\tsigned\t[%d:0]\tl;\n' % (B_location))

f.write('always@(*) begin\n')
f.write('\tcase(r)\n')
for i in range(np.size(GF67_positive)):
    f.write("\t\t%d: l = %+d;\n" % (GF67_positive[i] ,plus_values67[i]))
for j in range(np.size(GF67_negative)):
    f.write("\t\t%d: l = %+d;\n" % (GF67_negative[j] ,minus_values67[j]))
f.write("\t\tdefault: l = 0;\n")
f.write('\tendcase\n')
f.write('end\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file SECdecoder_location_24bits.v is generated.

